In [77]:
# imports
import pandas as pd
import random

import os

import numpy as np

In [78]:
# read in decklist
# WE ARE RUNNING 21 SWAMPS

decklist = pd.read_csv(os.path.join('data', 'decklist.csv'))

sub_ndxs = [0,1,2,3]

decklist.iloc[sub_ndxs,:]

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,name,mana_cost,cmc,colors,type_line,oracle_text,produced_mana,must_keeps
0,0,0,77,"Shizo, Death's Storehouse",NaN,0.0,[],Legendary Land,"{T}: Add {B}.\r\n{B}, {T}: Target legendary cr...",['B'],True
1,1,1,497,Diabolic Intent,{1}{B},2.0,['B'],Sorcery,"As an additional cost to cast this spell, sacr...",NaN,False
2,2,2,533,Misinformation,{B},1.0,['B'],Instant,Put up to three target cards from an opponent'...,NaN,False
3,3,3,610,Deadly Rollick,{3}{B},4.0,['B'],Instant,"If you control a commander, you may cast this ...",NaN,False


In [79]:
# add 

decklist['must_keeps'] = False

decklist.loc[decklist['type_line'].str.find('Land') != -1.0, 'must_keeps'] = True

decklist.to_csv(os.path.join('data', 'decklist.csv'))



In [80]:
decklist[decklist['name'] == 'Swamp']

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,name,mana_cost,cmc,colors,type_line,oracle_text,produced_mana,must_keeps
36,36,36,11592,Swamp,NaN,0.0,[],Basic Land — Swamp,({T}: Add {B}.),['B'],True


In [81]:
def gen_sampling_ndxs(deck_ndxs:list[int], no_must_keeps:int, no_basics:int) -> list[int]:
    
    return random.sample(deck_ndxs, 99-no_must_keeps-no_basics-1)

def decklist_from_ndxs(decklist:pd.DataFrame, ndxs:list[int]) -> pd.DataFrame:
    return decklist.iloc[ndxs, :]

def add_repeats(repeats:dict, deck_ndxs:list[int]) -> list[int]:
    for key, value in repeats.items():
        deck_ndxs = sum(
            [[key] * value, deck_ndxs], []
        )
    return deck_ndxs


def sample_decklist(decklist, repeats:dict) -> list[int]:

    """Collect a random sampling of your decklist
    decklist: a pandas dataframe representing your decklist
        in a scryfall format
    repeats: a dictionary of specific decklist ndxs 
        of cards you want repeated
        ! in a future update this will be collected 
            from the number attribute in the decklist"""
    

    # drop must keeps from sampling pool
    # and generate list of ndxs

    ndxs = decklist[decklist['must_keeps'] == False].index.tolist()
    
    # randomly sample the deck based on 
    # the number of basic lands and 'must keep' cards

    ndxs = gen_sampling_ndxs(
        deck_ndxs = ndxs, 
        no_must_keeps = decklist[decklist['must_keeps'] == True].shape[0],
        no_basics = 21 # we can solve for this 
    )
    
    # add in 'must keep' cards

    ndxs = sum(
        [decklist[decklist['must_keeps'] == True].index.tolist(), ndxs] , [] 
    )

    # add in cards that have multiples

    ndxs = add_repeats(repeats, ndxs)

    return ndxs




In [82]:
# sample the following from each decklist

## initial 7 cards
## free mulligan
## mulligan to 6
## mulligan to 5

# for the previous 4 situations draw two cards from the top of the library

# just deck things

In [83]:
def draw(deck_ndxs:list[int], no_cards:int) -> list[int]:
    return deck_ndxs[0:no_cards+1]

def shuffle(deck_ndxs:list[int]) -> list[int]:
    random.shuffle(deck_ndxs) # does not return original list
    return deck_ndxs

In [84]:
repeats = {
    # decklist ndx : no_repeats
    36 : 22
}

# save index sets as a parquet from a dataframe

draw(shuffle(sample_decklist(decklist, repeats)), 7)

[108, 95, 69, 39, 60, 93, 36, 36]

# sampling format

# generating samples

In [ ]:
# constants

no_deck_samples = 1
no_samples_per_deck = 1000 # we will eventually do this to stability
no_mulligans = 3 
no_cards_in_hand = 7
cards_drawn = 2

# get sampling pool

deck = shuffle(sample_decklist(decklist, repeats))

# sample cards by
# shuffling the deck and drawing 9 cards
#   where the first 7 are the initial hand 
#   and the last two are your first two draws

samples = [
    draw(shuffle(deck), no_cards_in_hand+cards_drawn-1) 
    for i in range(no_samples_per_deck*no_mulligans)
]

# generate sampling indices

sampling_multidex = sum(sum([
    [[(m, n, i) for i in range(no_mulligans)]
    for n in range(no_samples_per_deck)]
    for m in range(no_deck_samples)
], []), [])

# generate data column names

column_names = sum(
    [
        [f'card_no_{no}' for no in range(no_cards_in_hand)],
        [f'draw_no_{no}' for no in range(cards_drawn)]
    ], []
)

data = pd.DataFrame(
    samples, 
    index=pd.MultiIndex.from_tuples(
        sampling_multidex, names=['deck_sample', 'deck_sample_no','mulligan_count']
    ),
    columns=column_names
)

# data.to_parquet(os.path.join(
#     'data', 'samples.parquet'
# ))

data

card_no_0  card_no_1  card_no_2  \
deck_sample deck_sample_no mulligan_count                                    
0           0              0                      36         48         47   
                           1                      36         38         36   
                           2                      15         10         36   
            1              0                       4         35         80   
                           1                      21        105         67   
...                                              ...        ...        ...   
            998            1                      69         35         97   
                           2                      99         35         36   
            999            0                      51        103         97   
                           1                      60         43         13   
                           2                      96         36        106   

                                           card_no_3  card_no_4  card_no_5  \
deck_sample deck_sample_no mulligan_count                                    
0           0              0                      36         60         83   
                           1                      28         36         68   
                           2                      80         36         73   
            1              0                      33         36        113   
                           1                      36         74         81   
...                                              ...        ...        ...   
            998            1                      36         89         41   
                           2                     102         13         36   
            999            0                      36         36         36   
                           1                     113         47         32   
                           2                      36         29         33   

                                           card_no_6  draw_no_0  draw_no_1  
deck_sample deck_sample_no mulligan_count                                   
0           0              0                     105         36         59  
                           1                      37         59         29  
                           2                      61          7          4  
            1              0                      36         36        106  
                           1                     113         36        114  
...                                              ...        ...        ...  
            998            1                      36         36         80  
                           2                      43         21         36  
            999            0                      36          6         74  
                           1                       9         97         36  
                           2                      81         47         16  

[3000 rows x 9 columns]

In [86]:
pd.read_parquet(os.path.join('data','samples.parquet'))

card_no_0  card_no_1  card_no_2  \
deck_sample deck_sample_no mulligan_count                                    
0           0              0                      36         48         47   
                           1                      36         38         36   
                           2                      15         10         36   
            1              0                       4         35         80   
                           1                      21        105         67   
...                                              ...        ...        ...   
            998            1                      69         35         97   
                           2                      99         35         36   
            999            0                      51        103         97   
                           1                      60         43         13   
                           2                      96         36        106   

                                           card_no_3  card_no_4  card_no_5  \
deck_sample deck_sample_no mulligan_count                                    
0           0              0                      36         60         83   
                           1                      28         36         68   
                           2                      80         36         73   
            1              0                      33         36        113   
                           1                      36         74         81   
...                                              ...        ...        ...   
            998            1                      36         89         41   
                           2                     102         13         36   
            999            0                      36         36         36   
                           1                     113         47         32   
                           2                      36         29         33   

                                           card_no_6  draw_no_0  draw_no_1  
deck_sample deck_sample_no mulligan_count                                   
0           0              0                     105         36         59  
                           1                      37         59         29  
                           2                      61          7          4  
            1              0                      36         36        106  
                           1                     113         36        114  
...                                              ...        ...        ...  
            998            1                      36         36         80  
                           2                      43         21         36  
            999            0                      36          6         74  
                           1                       9         97         36  
                           2                      81         47         16  

[3000 rows x 9 columns]

In [88]:
# need to add a way to map card types or whatever to the integers

def get_mappings(decklist:pd.DataFrame, column:str) -> dict:

    return decklist.loc[:,column].to_dict()


for col in data.columns:
    data[col] = data[col].map(get_mappings(decklist, 'type_line'))

data

card_no_0  \
deck_sample deck_sample_no mulligan_count                                     
0           0              0                             Basic Land — Swamp   
                           1                             Basic Land — Swamp   
                           2               Legendary Creature — Human Noble   
            1              0                                           Land   
                           1                                        Instant   
...                                                                     ...   
            998            1                                    Land — Gate   
                           2                   Artifact — Equipment // Land   
            999            0                   Creature — Snake Ninja Rogue   
                           1                        Creature — Elder Dragon   
                           2                           Artifact — Equipment   

                                                                       card_no_1  \
deck_sample deck_sample_no mulligan_count                                          
0           0              0                                        Land — Swamp   
                           1               Legendary Creature — Vampire Assassin   
                           2                                Artifact — Equipment   
            1              0                                         Enchantment   
                           1               Legendary Creature — Elemental Shaman   
...                                                                          ...   
            998            1                                         Enchantment   
                           2                                         Enchantment   
            999            0                                  Enchantment — Aura   
                           1                                Artifact — Equipment   
                           2                                  Basic Land — Swamp   

                                                                       card_no_2  \
deck_sample deck_sample_no mulligan_count                                          
0           0              0                                            Artifact   
                           1                                  Basic Land — Swamp   
                           2                                  Basic Land — Swamp   
            1              0                                             Sorcery   
                           1                      Legendary Artifact — Equipment   
...                                                                          ...   
            998            1               Legendary Creature — Phyrexian Zombie   
                           2                                  Basic Land — Swamp   
            999            0               Legendary Creature — Phyrexian Zombie   
                           1                                             Instant   
                           2                                Artifact — Equipment   

                                                             card_no_3  \
deck_sample deck_sample_no mulligan_count                                
0           0              0                        Basic Land — Swamp   
                           1               Creature — Phyrexian Horror   
                           2                                   Sorcery   
            1              0                                   Instant   
                           1                        Basic Land — Swamp   
...                                                                ...   
            998            1                        Basic Land — Swamp   
                           2                                  Artifact   
            999            0                        Basic Land — Swamp   
                           1          